# Collecting and Analyzing Tweets

To replicate my set up and run this notebook, please visit https://github.com/galletti94/Tweet-Monitor 


## Exploratory Data Analysis

I love exploring data! It is such an important first step because, as you will see, it gives us intuition about what models may or may not work and why, when we try learning from the data later.

### Let's get some tweets!

For this step we use the twitter API which requires you to have a twitter account. Twitter gives you access keys which are unique to your account. These keys will go in a file called keys.txt which we make sure to include in the .gitignore file so that these are not made publically available. Let's get started!

In [13]:
import tweepy
import json
from pymongo import MongoClient
from __future__ import print_function

MONGO_HOST = 'mongodb://localhost/usa_db'

LOCATION = [-127.3,24.1,-65.9,51.8]

# get credentials from the keys.txt file
keys_file = open("keys.txt")
lines = keys_file.readlines()
consumer_key = lines[0].rstrip()
consumer_secret = lines[1].rstrip()
access_token = lines[2].rstrip()
access_token_secret = lines[3].rstrip()
keys_file.close()

class StreamListener(tweepy.StreamListener):
    # This is a class provided by tweepy to access the Twitter Streaming API.

    def on_connect(self):
        # Called initially to connect to the Streaming API
        print("You are now connected to the streaming API.")

    def on_error(self, status_code):
        print('An Error has occured: ' + repr(status_code))
        return False

    def on_data(self, data):
        try:
            client = MongoClient(MONGO_HOST)
            db = client.usa_db
            
            datajson = json.loads(data) # Decode the JSON from Twitter

            # grab the 'created_at' data from the Tweet to use for display
            created_at = datajson['created_at']
            
            # only get tweets that have geo location enabled
            if datajson['coordinates']:
                # print out a message to the screen that we have collected a tweet
                print("Tweet collected at " + str(created_at))
                db.usa_tweets_collection.insert_one(datajson) #insert into db
                
        except Exception as e:
            print(e)


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True))
streamer = tweepy.Stream(auth=auth, listener=listener)

print("Tracking: " + 'United States')
#streamer.filter(locations=LOCATION)

Tracking: United States


Awesome - now we can let this run and collect some tweets!

After waiting a few minutes for your database to fill up, you can hit CTRL + C to stop the collection and stop the program.

### What do we have here?

Let's take a look at the tweets we collected!

### Where are in the US are these tweets coming from?

We can use folium to map the coordinates of the tweets on a map:

In [5]:
from pymongo import MongoClient
import folium

client = MongoClient('localhost', 27017)
db = client['usa_db']
collection = db['usa_tweets_collection']
tweets_iterator = collection.find()

mymap = folium.Map(location=[45.372, -121.6972], zoom_start=4)

for tweet in tweets_iterator:
    folium.CircleMarker(location=list(reversed(tweet['coordinates']['coordinates']))).add_to(mymap)
    
mymap.save('map.html')

Running the above and opening map.html in a browser gives the following:

![Image](./tweetsUSA.jpg)

Nice. Let's look at what kinds of emojis people use!

### Emojis


In [7]:
from emoji import UNICODE_EMOJI

client = MongoClient('localhost', 27017)
db = client['usa_db']
collection = db['usa_tweets_collection']
tweets_iterator = collection.find()

d = dict()
i = 0
for tweet in tweets_iterator:
  for ch in list(tweet['text']):  #remember emojis are characters not words
    if ch in UNICODE_EMOJI:
      try:
        d[ch] += 1
      except KeyError:
        d[ch] = 1

d = sorted(d.items(), key=lambda x: -x[1])
print(d[:15])

[('❤', 164), ('🔥', 160), ('🎄', 121), ('😍', 118), ('🏾', 112), ('🏼', 112), ('😂', 104), ('🏻', 90), ('🏽', 81), ('✨', 79), ('🙌', 68), ('💪', 67), ('💯', 62), ('🙏', 60), ('❄', 60)]


When this was written it was close to christmas so I guess the second and third emojis make sense.


### Sentiment Analysis


We can analyze the words of the tweets to get a sense of whether the tweet has positive sentiment or negative sentiment. We can aggregate these and get a sense of the general mood of the US at this time of year.

In [5]:
from pymongo import MongoClient
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

client = MongoClient('localhost', 27017)
db = client['usa_db']
collection = db['usa_tweets_collection']
tweets_iterator = collection.find()

tweetCnt = 0
locEnabled = 0
d = dict()
only100 = 100
for tweet in tweets_iterator:
    if only100 <= 0:
        break
    only100 -= 1
    tweetCnt += 1
    if tweet['user']['location']:
      locEnabled += 1
    
    blob = TextBlob(tweet['text'], analyzer=NaiveBayesAnalyzer())
    try:
        d[blob.sentiment.classification] += 1
    except KeyError:
        d[blob.sentiment.classification] = 1

d = sorted(d.items(), key=lambda x: -x[1])
print(d)
print()
print('Number of Tweets = ', tweetCnt)
print()
print('Number of tweets with location enabled = ', locEnabled)

[('pos', 74), ('neg', 26)]

Number of Tweets =  100

Number of tweets with location enabled =  95


Seems like a majority of tweets have positive sentiment but this analysis was only conducted on a subset of the data collected since sentiment analysis is expensive to compute.

Maybe next we can see which emojis are correlated with which sentiment!

### Emoji vs Sentiment


In [3]:
from pymongo import MongoClient
from emoji import UNICODE_EMOJI
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

client = MongoClient('localhost', 27017)
db = client['usa_db']
collection = db['usa_tweets_collection']
tweets_iterator = collection.find()

pos_dict = dict()
neg_dict = dict()
only200 = 200
for tweet in tweets_iterator:
    if only200 <= 0:
        break
    for ch in list(tweet['text']):
        if ch in UNICODE_EMOJI:
            only200 -= 1
            blob = TextBlob(tweet['text'], analyzer=NaiveBayesAnalyzer())
            if blob.sentiment.classification == 'pos':
                try:
                    pos_dict[ch] += 1
                except KeyError:
                    pos_dict[ch] = 1
            else:
                try:
                    neg_dict[ch] += 1
                except KeyError:
                    neg_dict[ch] = 1

pos_dict = sorted(pos_dict.items(), key=lambda x: -x[1])
neg_dict = sorted(neg_dict.items(), key=lambda x: -x[1])                    
print('positive emojis = ', pos_dict[:5])
print()
print('negative_emojis = ', neg_dict[:5])

positive emojis =  [('🌹', 9), ('❤', 9), ('✨', 8), ('🎄', 6), ('🎶', 6)]

negative_emojis =  [('😭', 7), ('🔥', 6), ('💦', 4), ('❣', 2), ('⛓', 2)]


Great! It seems intuitive that a rose, a heart, and stars be associated with positive sentiment and a crying face would be associated with negative sentiment. It is good to see these results support our intuition!

Now we would like to know what kinds of tweets do people with location disabled make?

### Location Disabled Tweets - Emojis

First let's look at the kinds of emojis are used in both groups.

In [14]:
from pymongo import MongoClient
from emoji import UNICODE_EMOJI

client = MongoClient('localhost', 27017)
db = client['usa_db']
collection = db['usa_tweets_collection']
tweets_iterator = collection.find()

enabled_dict = dict()
disabled_dict = dict()
tweetcnt = 0
for tweet in tweets_iterator:
    tweetcnt += 1
    for ch in list(tweet['text']):
        if ch in UNICODE_EMOJI:
            if tweet['coordinates']:
                try:
                    enabled_dict[ch] += 1
                except KeyError:
                    enabled_dict[ch] = 1
            else:
                try:
                    disabled_dict[ch] += 1
                except KeyError:
                    disabled_dict[ch] = 1

enabled_dict = sorted(enabled_dict.items(), key=lambda x: -x[1])
disabled_dict = sorted(disabled_dict.items(), key=lambda x: -x[1])
print('Number of Tweets = ', tweetcnt)
print()
print('location enabled emojis = ', enabled_dict[:10])
print()
print('location disabled emojis = ', disabled_dict[:10])

Number of Tweets =  23226

location enabled emojis =  [('❤', 169), ('🔥', 164), ('🎄', 124), ('😍', 122), ('🏼', 115), ('🏾', 114), ('😂', 104), ('🏻', 95), ('🏽', 93), ('✨', 81)]

location disabled emojis =  [('😂', 125), ('😭', 81), ('♀', 28), ('🏾', 28), ('🏽', 28), ('😍', 27), ('\U0001f926', 26), ('❤', 26), ('🔥', 25), ('❄', 25)]


Looks like people with location disabled use in majority emojis that we evaluated earlier to be negative sentiment! Now that's very interesting!

I wonder if there is a difference in what topics these two groups post about. Let's look at the tags in their tweets to get a sense of what these people post about!

### Location Disabled Tweets - Tags

In [1]:
from pymongo import MongoClient
import re

client = MongoClient('localhost', 27017)
db = client['usa_db']
collection = db['usa_tweets_collection']
tweets_iterator = collection.find()

enabled_dict = dict()
disabled_dict = dict()
tweetcnt = 0
for tweet in tweets_iterator:
    tweetcnt += 1
    for tag in re.findall(r'\B(\#[a-zA-Z]+\b)(?!;)', tweet['text']):
        if tweet['coordinates']:
            try:
                enabled_dict[tag] += 1
            except KeyError:
                enabled_dict[tag] = 1
        else:
            try:
                disabled_dict[tag] += 1
            except KeyError:
                disabled_dict[tag] = 1

enabled_dict = sorted(enabled_dict.items(), key=lambda x: -x[1])
disabled_dict = sorted(disabled_dict.items(), key=lambda x: -x[1])
print('Number of Tweets = ', tweetcnt)
print()
print('location enabled tags = ', enabled_dict[:10])
print()
print('location disabled tags = ', disabled_dict[:10])

Number of Tweets =  23226

location enabled tags =  [('#job', 7322), ('#CareerArc', 6157), ('#Hiring', 5750), ('#hiring', 2842), ('#Jobs', 1996), ('#Job', 1995), ('#Retail', 1040), ('#Hospitality', 750), ('#Sales', 460), ('#Nursing', 433)]

location disabled tags =  [('#snow', 2), ('#ImpeachTrump', 2), ('#TheResistance', 2), ('#netneutrality', 2), ('#PokemonGO', 2), ('#dahlia', 1), ('#SugarLandTX', 1), ('#loser', 1), ('#BetterTogether', 1), ('#GoHawks', 1)]


It looks like tweets with location disabled don't tag very much. I will let you make your own conclusions about the content type...

Let's start from a specific set of keywords and observe the distribution of location enabled vs disabled!

### Keywords about Machine Learning

In [8]:
from __future__ import print_function
import tweepy
import json
from pymongo import MongoClient

MONGO_HOST = 'mongodb://localhost/twitterdb'

WORDS = ['#deeplearning', '#computervision', '#datascience', '#bigdata']

keys_file = open("keys.txt")
lines = keys_file.readlines()
consumer_key = lines[0].rstrip()
consumer_secret = lines[1].rstrip()
access_token = lines[2].rstrip()
access_token_secret = lines[3].rstrip()

class StreamListener(tweepy.StreamListener):
    def on_connect(self):
        print("You are now connected to the streaming API.")

    def on_error(self, status_code):
        print('An Error has occured: ' + repr(status_code))
        return False

    def on_data(self, data):
        # This is the meat of the script...it connects to your mongoDB and stores the tweet
        try:
            client = MongoClient(MONGO_HOST)
            db = client.twitterdb
            datajson = json.loads(data)
            created_at = datajson['created_at']
            print("Tweet collected at " + str(created_at))
            db.twitter_search.insert(datajson)
        except Exception as e:
            print(e)

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
listener = StreamListener(api=tweepy.API(wait_on_rate_limit=True))
streamer = tweepy.Stream(auth=auth, listener=listener)

print("Tracking: " + str(WORDS))
streamer.filter(track=WORDS)

Tracking: ['#deeplearning', '#computervision', '#datascience', '#bigdata']


As before, keep this runnning for a bit while to collect some tweets. Now we can do some more analysis:

### Location of ML people

In [18]:
from pymongo import MongoClient
from emoji import UNICODE_EMOJI

client = MongoClient('localhost', 27017)
db = client['twitterdb']
collection = db['twitter_search']
tweets_iterator = collection.find()

d = dict()
enabled = 0
disabled = 0
tweetcnt = 0
for tweet in tweets_iterator:
    tweetcnt += 1
    
    for ch in list(tweet['text']):  #remember emojis are characters not words
        if ch in UNICODE_EMOJI:
          try:
            d[ch] += 1
          except KeyError:
            d[ch] = 1
    
    if tweet['coordinates']:
        enabled += 1
    else:
        disabled += 1

d = sorted(d.items(), key=lambda x: -x[1])
print('Total Tweets = ', tweetcnt)
print('location enabled = ', enabled)
print('location disabled = ', disabled)
print('emojis = ', d)

Total Tweets =  1019
location enabled =  0
location disabled =  1019
emojis =  [('🗣', 20), ('👏', 8), ('👉', 6), ('👀', 1), ('™', 1), ('📦', 1), ('📢', 1), ('🐸', 1), ('📣', 1), ('▶', 1), ('🏽', 1), ('®', 1), ('📈', 1), ('🚨', 1), ('🌐', 1)]


Looks like people who post about these keywords don't share their location! Interesting emojis as well!

Going back to USA tweets, we can analyze the source of the tweet to understand what people post about on which device for example. First we will look at the distribution of devices, then we can plot these and finally look at what emojis and tags are used with each.

### Source of Tweets

In [3]:
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['usa_db']
collection = db['usa_tweets_collection']
tweets_iterator = collection.find()

d = dict()
for tweet in tweets_iterator:
    s = ''.join(tweet['source'].split('>')[1].split('<')[0])
    if s:
        try:
            d[s] += 1
        except KeyError:
            d[s] = 1

d = sorted(d.items(), key=lambda x: -x[1])
print(d[:10])

[('TweetMyJOBS', 9585), ('Instagram', 6765), ('Twitter for iPhone', 1387), ('Foursquare', 881), ('SafeTweet by TweetMyJOBS', 665), ('NYRegions', 392), ('Cities', 360), ('Twitter for Android', 324), ('TownTweet', 300), ('Untappd', 140)]


Let's map some of these!

### Map of devices

In [7]:
from pymongo import MongoClient
import folium

client = MongoClient('localhost', 27017)
db = client['usa_db']
collection = db['usa_tweets_collection']
tweets_iterator = collection.find()

mymap = folium.Map(location=[45.372, -121.6972], zoom_start=4)

for tweet in tweets_iterator:
    if ''.join(tweet['source'].split('>')[1].split('<')[0]) == 'TweetMyJOBS' and tweet['coordinates']:
        folium.CircleMarker(location=list(reversed(tweet['coordinates']['coordinates']))).add_to(mymap)
    
mymap.save('map3.html')

![Image](./tweetsUSA2.jpg)

It is interesting to note that many people who use this feature have their location enabled. If we now plot the tweets made on the Twitter App for iPhone, the results are drastically different. Are your location settings application dependent? That is, if you share an instagram post on twitter, is your location setting that of instagram or that of twitter? Someone should test this out!

![Image](./tweetsUSA3.jpg)

Stay tuned for more!